In [2]:
import os
import serial


In [3]:
s = serial.Serial(port='/dev/ttyUSB1', baudrate=115200, timeout=0.1)

In [4]:
import serial.tools.list_ports

# Get a list of all available serial ports
available_ports = serial.tools.list_ports.comports()

# Define the pattern to search for in the port description
pattern = "Novatel"

# Declare variables to connect to the first two devices with matching descriptions
device1 = None
device2 = None

# Loop through the list of available ports and look for matching descriptions
for port in available_ports:
    if pattern in port.usb_description():
        if device1 is None:
            device1 = serial.Serial(port.device)
        elif device2 is None:
            device2 = serial.Serial(port.device)
        if device1 is not None and device2 is not None:
            break

# Print out the device names (if found)
if device1 is not None:
    print("Connected to Novatel device 1 on", device1.name)
if device2 is not None:
    print("Connected to Novatel device 2 on", device2.name)


Connected to Novatel device 1 on /dev/ttyUSB2
Connected to Novatel device 2 on /dev/ttyUSB1


In [5]:
def read_port_formatted(port : serial.Serial) -> str:
    '''
        Reads the serial port for its current value\n
        Additionally removes last escape characters from string
    '''
    if port is None: raise Exception("Port undefined")
    
    resp = port.readline().decode('utf-8')
    return resp[:len(resp) - 2]


def write_without_response(data : str, port : serial.Serial):
    '''
        Writes a string to the serial port without waiting for response
    '''
    if port is None: raise Exception("Port undefined")
    port.write(data.encode('utf-8'))

In [29]:
write_without_response("log bestpos ontime 0.25\r\n", port=device1)
# write_without_response("log heading ontime 0.25\r\n", port=device2)
#https://docs.novatel.com/OEM7/Content/Logs/HEADING2.htm?Highlight=heading


In [10]:
import re

def is_valid_decimal(value):
    return re.match(r'^-?\d+(\.\d+)?$', value)

def read_gps_port(port):
    data = read_port_formatted(port)
    msg_split = data.split(" ")

    if len(msg_split) < 9 or not (is_valid_decimal(msg_split[7]) and is_valid_decimal(msg_split[8])):
        return None

    msg_dict = {'longitude': msg_split[8], 'latitude': msg_split[7]}
    return msg_dict


In [27]:
import math

def decimal_degrees(degrees_minutes):
    degrees = int(degrees_minutes // 100)
    minutes = degrees_minutes % 100
    return degrees + (minutes / 60)

def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1

    y = math.sin(dlon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dlon)

    bearing = math.atan2(y, x)
    bearing = math.degrees(bearing)
    bearing = (bearing + 360) % 360
    return bearing


In [35]:
lat_old = None
lon_old = None
while True:
    gps_dict1 = read_gps_port(device1)
    if gps_dict1:
        print(gps_dict1)
        lat_new = float(gps_dict1['latitude'])
        lon_new = float(gps_dict1['longitude'])

        #gps_dict2 = read_gps_port(device2)
        #print("2",gps_dict2)
        #lat2 = float(gps_dict2['latitude'])
        #lon2 = float(gps_dict2['longitude'])
        if lat_old and lon_old and lat_new and lon_new:
            print(lat_new,lon_new,lat_old,lon_old)
            print(calculate_bearing(lat_new,lon_new,lat_old,lon_old))
        lat_old = lat_new
        lon_old = lon_new

{'longitude': '-84.51689147567', 'latitude': '39.13299018258'}
{'longitude': '-84.51689148852', 'latitude': '39.13299051026'}
39.13299051026 -84.51689148852 39.13299018258 -84.51689147567
178.25768645206824
{'longitude': '-84.51689156539', 'latitude': '39.13299016057'}
39.13299016057 -84.51689156539 39.13299051026 -84.51689148852
9.676621486770614
{'longitude': '-84.51689174515', 'latitude': '39.13298973344'}
39.13298973344 -84.51689174515 39.13299016057 -84.51689156539
18.079297499299287
{'longitude': '-84.51689191352', 'latitude': '39.13298913338'}
39.13298913338 -84.51689191352 39.13298973344 -84.51689174515
12.278809084472357
{'longitude': '-84.51689196292', 'latitude': '39.13298920735'}
39.13298920735 -84.51689196292 39.13298913338 -84.51689191352
152.6144485476109
{'longitude': '-84.51689212551', 'latitude': '39.13298847518'}
39.13298847518 -84.51689212551 39.13298920735 -84.51689196292
9.773448596368041
{'longitude': '-84.51689236544', 'latitude': '39.13298721222'}
39.1329872122

KeyboardInterrupt: 

In [16]:
write_without_response("unlogall\r\n", port=s)

In [15]:
write_without_response("log gpgga ontime 0.25\r\n", port=device2)


In [16]:
import re

def parse_gpgga(sentence):
    gpgga_pattern = re.compile(
        r"\$GPGGA,\d+\.\d+,"  # Time
        r"(\d+\.\d+),([NS]),"  # Latitude
        r"(\d+\.\d+),([EW]),"  # Longitude
        r".*")  # Rest of the sentence

    match = gpgga_pattern.match(sentence)
    if not match:
        return None

    lat, lat_hemi, lon, lon_hemi = match.groups()
    lat = float(lat)
    lon = float(lon)

    lat_deg = decimal_degrees(lat)
    lon_deg = decimal_degrees(lon)

    if lat_hemi == "S":
        lat_deg = -lat_deg

    if lon_hemi == "W":
        lon_deg = -lon_deg

    return lat_deg, lon_deg


In [17]:
while True:
    data1 = read_port_formatted(device2)
    lat1, lon1 = parse_gpgga(data1)

    data2 = read_port_formatted(device2)
    lat2, lon2 = parse_gpgga(data2)

    bearing = calculate_bearing(lat1, lon1, lat2, lon2)
    print(data1)
    print(data2)
    print(bearing)
    

TypeError: cannot unpack non-iterable NoneType object

In [18]:
while True:
    read_port_formatted(device2)

KeyboardInterrupt: 